<font style='color:red'>CH3. 연관분석</font>
- pip install apyori

# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석
- 활용분야 : 이벤트 미리 감지(사기적발, ...), 신상품 카테고리 구성

[조건 : left-hand side : 오렌지주스] ->[ 결과 : right-hand side : 와인]
1. 지지도(support) : 얼마나 자주 함께 나타나는지
    (lhs와 rhs)의 항목수 / 전체항목수 = 0.2(오렌지주스 계산한 값)
2. 신뢰도(confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지
    (lhs_>rhs)의 항목수/lhs의 항목수 = 1/2 = 0.5
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인
    lhs->rhs의 항목 / (lhs의 지지도*rhs의 항목수)
    => 0.2 / (0.4*0.6) = 0.833
    향상도 < 1 : 기대가 낮다
    향상도 > 1 : 기대가 높다 (높을수록 둘의 관계가 높음)

# 2. 연관분석 구현

In [2]:
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [ ]:
from apyori import apriori
rules = apriori(transaction, # 2차원 데이터
                min_support=0.15,
                min_confidence=0.1)

rules = list(rules)
len(rules)

18

In [9]:
rules[10]

RelationRecord(items=frozenset({'콜라', '소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '소주'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [21]:
rule = rules[10]
support = rule[1]
order_st = rule[2]
for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence = item[2]
    lift = item[3]
    if lift > 1:
        print('{}=>{}\t {}\t {:.2}\t {:.3}'.format(lhs,rhs,support, confidence, lift))

frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25


In [23]:
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = item[0]
        rhs = item[1]
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            print('{}=>{}\t {}\t {:.2}\t {:.3}'.format(lhs,rhs,support, confidence, lift))


frozenset({'맥주'})=>frozenset({'콜라'})	 0.4	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'맥주'})	 0.4	 0.5	 1.25
frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25
frozenset({'콜라'})=>frozenset({'맥주', '소주'})	 0.2	 0.25	 1.25
frozenset({'맥주', '소주'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'맥주'})=>frozenset({'콜라', '와인'})	 0.2	 0.5	 1.25
frozenset({'콜라'})=>frozenset({'맥주', '와인'})	 0.2	 0.25	 1.25
frozenset({'맥주', '와인'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'콜라', '와인'})=>frozenset({'맥주'})	 0.2	 0.5	 1.25
frozenset({'소주'})=>frozenset({'콜라', '오렌지주스'})	 0.2	 0.33	 1.67
frozenset({'콜라'})=>frozenset({'소주', '오렌지주스'})	 0.2	 0.25	 1.25
frozenset({'소주', '오렌지주스'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25
frozenset({'콜라', '오렌지주스'})=>frozenset({'소주'})	 0.2	 1.0	 1.67
frozenset({'콜라'})=>frozenset({'소주', '와인'})	 0.2	 0.25	 1.25
frozenset({'소주', '와인'})=>frozenset({'콜라'})	 0.2	 1.0	 1.25


In [37]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support', 'confidence', 'lift'])
i = 0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join(item[0])
        rhs = ', '.join(item[1])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[i] = [lhs,rhs,support, round(confidence,2), round(lift,2)]
            i += 1
rules_df.sort_values(['lift','confidence'], ascending=False)

,lhs,rhs,support,confidence,lift
13,"콜라, 오렌지주스",소주,0.2,1.00,1.67
10,소주,"콜라, 오렌지주스",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주, 소주",콜라,0.2,1.00,1.25
8,"맥주, 와인",콜라,0.2,1.00,1.25
12,"소주, 오렌지주스",콜라,0.2,1.00,1.25
15,"소주, 와인",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 경주/전주 여행 자료 연관분석

In [43]:
import pandas as pd
df = pd.read_csv('data/전경주.csv')
total_text_list = list(df['total_text'])
total_text_list

['전주 가볼만한곳 추천 받아요 추억의 7080 다양한체험 7080감성 추억여행 테마박물관 유익한시간 2 전북 전북투어패스 통합이용권 전북핫플 여러여행지 다양한체험 카페이용추가 전주여행필수 편안하고 즐거운 날이 되시길 바라겠습니다 감사합니다 ',
 '전주여행갈려고하는데요!전주여행을 갈려고하는데요 아는사람과 갈려고하는데 호텔은 좋은가격에 정했고 음 2박3일여행인데 얼마정도갖고가면좋을까요 그리고 맛집같은거 카페같은거 추천해주세요 안녕하세요 전주 여행 계획 중이시네요 한옥마을 근처 ',
 '전주 1박 2일 여행 관련 질문~ 전주 여행을 계획하고 계시다니 정말 좋은 선택을 하셨네요 전주는 한옥마을과 함께 멋진 자연경관 맛있는 음식이 가득한 동네이니 정말 즐거운 시간을 보낼 수 있을 거예요 우선 한옥마을 근처에서 가볼 수 있는 장소로는 ',
 '중2 여학생 친구와 전주여행안녕하세요 중 2 15살 여학생입니다 친한친구와 둘이서 전주 여행을 가기로 했는데 어디로 가야할지를 몰라서 너무 막막합니다 저희 둘다 일본 애니를 너무 좋아하는데전주에 오타쿠들이 갈만할 곳 있을까요 제발 알려주세요 ',
 '부모닝 모시고 전주여행부모님 모시고 전주여행가려는데 명소 식당 추천부탁드립니다 전주는 한옥마을로 유명한 완산구가 있으며 덕진구 또한 멋진 추억을 가진 장소가 많이 ',
 '2박3일 전주여행이번주 토 월 2박3일 여자친구와 전주여행을 가는데 여행경로 좀 알려주세요 토요일엔 비가 온다고 하는데 실내데이트 가능한곳도 알려주시면 감사하겠습니다 숙소는 전주한옥마을 근처에 잡았습니다 토 한옥마을 구경 후 ',
 '전주 1박2일 여행 저녁엔 전주천 산책길 따라 걷거나 한옥마을 근처에서 야경 즐기시는 것도 너무 좋답니다 혹시 일정이나 관심 있는 테마 사진 카페 체험 등 더 알려주시면 맞춤 추천도 해드릴 수 있어요 즐거운 전주 여행 되시길 바랄게요 ',
 '전주 여행코스 추천이요~!영화제 기간 일정으로 2박3일 전주여행 좀 가볼까 해서요 여행코스 추천 좀 해주세요 어차피 주목적은 전주국제영화제니까 

In [66]:
from konlpy.tag import Hannanum
from mecab import MeCab
total_noun_list = []
analyzer = Hannanum()
for total_text in total_text_list:
    total_noun_list.append([token for token, tag in analyzer.pos(total_text, ntags=22) 
                            if tag in ['NC','NQ'] and tag not in '여행' and len(token) > 1])
total_noun_list

[['전주',
  '가볼만한곳',
  '추천',
  '추억',
  '다양한체험',
  '7080감성',
  '추억여행',
  '테마박물관',
  '유익한시간',
  '전북투어패스',
  '통합이용권',
  '전북핫플',
  '여러여행지',
  '다양한체험',
  '카페이용추',
  '전주여행필수',
  '편안',
  '되시길',
  '감사'],
 ['전주여행갈려고하는데요',
  '전주여행',
  '아는사람',
  '호텔',
  '좋은가격',
  '2박3일여행인데',
  '얼마정도갖고가면좋을까요',
  '맛집같은거',
  '카페같은거',
  '추천',
  '안녕',
  '전주',
  '여행',
  '계획',
  '한옥마을',
  '근처'],
 ['전주',
  '여행',
  '관련',
  '질문',
  '전주',
  '여행',
  '계획하',
  '선택',
  '전주',
  '한옥마을',
  '자연경관',
  '음식',
  '가득',
  '동네',
  '시간',
  '거예요',
  '한옥마을',
  '근처',
  '장소'],
 ['중2',
  '여학생',
  '친구',
  '전주여행안녕',
  '여학생',
  '친한친구',
  '전주',
  '여행',
  '막막',
  '둘다',
  '애니',
  '좋아하는데전주',
  '오타쿠들'],
 ['부모닝',
  '전주여행부모님',
  '전주여행가려는데',
  '명소',
  '식당',
  '추천부탁드',
  '전주',
  '한옥마을',
  '유명',
  '완산구',
  '덕진구',
  '추억',
  '장소'],
 ['2박3',
  '전주여행이번주',
  '2박3',
  '여자친구',
  '전주여행',
  '여행경',
  '토요일',
  '실내데이트',
  '가능한곳',
  '감사',
  '숙소',
  '전주한옥마을',
  '근처',
  '한옥마을',
  '구경'],
 ['전주',
  '1박2',
  '여행',
  '저녁',
  '전주천',
  '산책길',
  '한옥마을',
  '근처',
  '야경',
  '일정',
  '

In [67]:
rules = apriori(total_noun_list, min_support=0.15, min_confidence=0.3)
rules = list(rules)
len(rules)

589

In [69]:
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support', 'confidence', 'lift'])
i = 0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join(item[0])
        rhs = ', '.join(item[1])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[i] = [lhs,rhs,support, round(confidence,2), round(lift,2)]
            i += 1
rules_df.sort_values(['lift','confidence'], ascending=False, inplace=True)

In [73]:
rules_df.head(40)

,lhs,rhs,support,confidence,lift
1041,"국내여행, 여수가볼만한곳","여수여행, 전주맛집",0.152,1.0,6.37
1045,"여수가볼만한곳, 전주맛집","국내여행, 여수여행",0.152,1.0,6.37
1101,"여수가볼만한곳, 전주맛집","국내여행, 전주가볼만한곳",0.152,1.0,6.37
1238,"국내여행, 전주가볼만한곳","여수여행, 전주맛집",0.157,1.0,6.37
1241,"여수여행, 전주맛집","국내여행, 전주가볼만한곳",0.157,1.0,6.37
2838,"국내여행, 여수가볼만한곳","달봉티비, 여수여행, 전주맛집",0.152,1.0,6.37
2845,"여수가볼만한곳, 전주맛집","국내여행, 달봉티비, 여수여행",0.152,1.0,6.37
2847,"국내여행, 달봉티비, 여수가볼만한곳","여수여행, 전주맛집",0.152,1.0,6.37
2854,"달봉티비, 여수가볼만한곳, 전주맛집","국내여행, 여수여행",0.152,1.0,6.37
2965,"여수가볼만한곳, 전주맛집","국내여행, 달봉티비, 전주가볼만한곳",0.152,1.0,6.37
